In [1]:
import sys
import numpy as np
import pandas as pd
import json

sys.path.append('../..')

from heat_load_calc.core import core

# core

## 1. Outline

モジュール core は建物仕様・外気温湿度...(後で詳細に記述）...を与えることで、ステップごとのループ計算を行い、室温度・湿度、暖冷房負荷を計算するモジュールです。

## 2. Interface

calc(input_data_dir, output_data_dir = None, show_simple_result = False, show_detail_result = False)

params:

| パラメータ名　| 型 | 概要 |
| :--- | :--- | :--- |
| input_data_dir | str | 入力データが格納されたディレクトリを指定します。 |
| output_data_dir | str | （省略可能）計算された室温度・湿度、暖冷房負荷等をCSVで出力するフォルダを指定します。CSVを出力しない場合は省略可能。 |
| show_simple_result | bool | （省略可能）計算結果（概要版）をはきだす場合はTrueにします。はきださない場合はFalseを指定しないか省略する。 |
| show_detail_result | bool | （省略可能）計算結果（詳細版）をはきだす場合はTrueにします。はきださない場合はFalseを指定しないか省略する。 |

returns:

計算結果の概要版と詳細版をpandas.DataFrame 形式で返します。  
2つの pandas.DataFrame 形式ファイルを、タプル形式で返します。  
（簡易版計算結果, 詳細版計算結果）

## 3. Input Files

引数 'input_data_dir' で指定したフォルダに以下のファイルを用意しないといけない。

### 1) mid_data_house.json

#### 仕様

json ファイルの仕様

In [2]:
pd.set_option('display.max_columns', 300)
pd.read_csv('../../docs/data_spec/spec_mid_data_house.csv').fillna('')

,1,2,3,4,5,6,7,key,項目名,必須,条件,型,単位,入力,備考
0,〇,,,,,,,building,建物,〇,,dict,－,,
1,,〇,,,,,,infiltration_method,隙間風の計算方法,〇,,str,－,{'balance_residential'},
2,,〇,,,,,,story,建物の階数,△,{'infiltration_method': 'balance_residential'},int,－,,1又は2。
3,,〇,,,,,,c_value,相当隙間面積,△,{'infiltration_method': 'balance_residential'},float,cm2/m2,,0以上。
4,,〇,,,,,,inside_pressure,室内圧力,△,{'infiltration_method': 'balance_residential'},str,－,"{'positive', 'negative', 'balanced'}",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,,,〇,,,,,f_mrt_hum,部位の人体に対する形態係数,〇,,float,－,,
67,,,〇,,,,,k_outside,隣室温度差係数（室外側相当外気温度基準）,〇,,float,－,,
68,,,〇,,,,,k_inside,隣室の影響,〇,,"dict, null",－,,隣室がない場合はk_insideがnullとなる。
69,,,,〇,,,,id,裏面の境界通し番号,△,,int,－,,k_insideがnullでない場合に定義される。


#### json ファイルの基本構成

基本的なjsonファイルの書き方は以下のとおり。

{  
    'spaces': [  
        {'id': 0, ...その他の情報...},  
        {'id': 1, ...その他の情報...},  
        ...  
        {'id': 10, ...その他の情報...}  
    ],       
    'boundaries': [
        {'id': 0, ...その他の情報...},  
        {'id': 1, ...その他の情報...},  
        ...  
        {'id': 100, ...その他の情報...}  
    ]  
} 

#### スペース1つ分の書き方の例

In [3]:
# スペース1つ分の書き方例
with open('core_data_tutorial/mid_data_house.json') as f:
    dj = json.load(f)

dj['spaces'][0]

{'id': 0,
 'name': 'main_occupant_room',
 'volume': 70.92,
 'c_value': 2.0,
 'beta': 0.0,
 'ventilation': {'mechanical': 0.016666666666666666,
  'next_spaces': [0.0, 0.0, 0.0],
  'natural': 0.0985},
 'furniture': {'heat_capacity': 893592.0,
  'heat_cond': 196.59024,
  'moisture_capacity': 1191.4560000000001,
  'moisture_cond': 2.1446208},
 'equipment': {'heating': {'radiative': {'installed': False},
   'convective': {}},
  'cooling': {'radiative': {'installed': False},
   'convective': {'q_min': 500.0,
    'q_max': 6049.891511,
    'v_min': 11.086608037753514,
    'v_max': 20.15746915955184}}}}

#### 境界1つ分の書き方の例

In [4]:
with open('core_data_tutorial/mid_data_house.json') as f:
    dj = json.load(f)

dj['boundaries'][0]

{'id': 0,
 'name': 'integrated_boundary0',
 'sub_name': 'north_exterior_wall',
 'is_ground': 'false',
 'connected_space_id': 0,
 'area': 4.9775,
 'phi_a0': 0.07082379236222458,
 'phi_a1': [4.507057985882911e-05,
  -0.0009266420480683226,
  0.015710747070284452,
  0.1371664403861094,
  -0.11163144218064561,
  0.09174099947400259,
  -0.022630779908299824,
  0.0033885297113725936,
  0.0,
  0.0,
  0.0,
  0.0],
 'phi_t0': 0.001598173949854509,
 'phi_t1': [4.014649140609802e-05,
  -0.0008192243722720102,
  0.013605804259968005,
  0.1088867059986907,
  -0.19238689071678203,
  0.10296731413100914,
  -0.022435410952413436,
  0.0029186857277683836,
  0.0,
  0.0,
  0.0,
  0.0],
 'r': [0.9982016190284373,
  0.99291521625209,
  0.9723062034557024,
  0.8950058718522789,
  0.6452076990440302,
  0.17714441059303657,
  0.0010735761574841769,
  1.869406715200079e-12,
  0.0,
  0.0,
  0.0,
  0.0],
 'h_i': 9.090909090909092,
 'flr': 0.0,
 'is_solar_absorbed': 'False',
 'f_mrt_hum': 0.027134347293875108,
 '

### 2) weather.csv

#### 仕様

In [5]:
pd.read_csv('../../docs/data_spec/d_weather.csv')

,列名,単位,概要
0,temperature,degree C,外気温度
1,absolute humidity,kg/kg(DA),外気絶対湿度
2,normal direct solar radiation,W/m2,法線面直達日射量
3,horizontal sky solar radiation,W/m2,水平面天空日射量
4,outward radiation,W/m2,夜間放射量
5,sun altitude,rad,太陽高度
6,sun azimuth,rad,太陽方位角


#### 記載例

In [6]:
pd.read_csv('core_data_tutorial/weather.csv').head()

,Unnamed: 0,temperature,absolute humidity,normal direct solar radiation,horizontal sky solar radiation,outward radiation,sun altitude,sun azimuth
0,1989-01-01 00:00:00,2.300,0.003200,0.0,0.0,83.333333,-1.365523,2.988844
1,1989-01-01 00:15:00,2.375,0.003225,0.0,0.0,82.638889,-1.365761,-2.997617
2,1989-01-01 00:30:00,2.450,0.003250,0.0,0.0,81.944444,-1.350635,-2.720263
3,1989-01-01 00:45:00,2.525,0.003275,0.0,0.0,81.250000,-1.322955,-2.488647
4,1989-01-01 01:00:00,2.600,0.003300,0.0,0.0,80.555556,-1.286386,-2.305989


### 3) mid_data_ac_demand.csv

#### 値

True：空調ON  
False：空調OFF

#### 例

In [7]:
pd.read_csv('core_data_tutorial/mid_data_ac_demand.csv', header=None).head()

,0,1,2
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False


### 4) mid_data_heat_generation.csv

#### 値

機器内部発熱値, W

#### 例

In [8]:
pd.read_csv('core_data_tutorial/mid_data_heat_generation.csv', header=None).head()

,0,1,2
0,135.0,0.867,44.7
1,135.0,0.867,44.7
2,135.0,0.867,44.7
3,135.0,0.867,44.7
4,10.3,0.867,44.7


### 5) mid_data_local_vent.csv

#### 値

局所換気量, m<sup>3</sup>/s

#### 例

In [9]:
pd.read_csv('core_data_tutorial/mid_data_local_vent.csv', header=None).head()

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


### 6) mid_data_moisture_generation.csv

#### 値

調理内部発湿, kg/s

#### 例

In [10]:
pd.read_csv('core_data_tutorial/mid_data_moisture_generation.csv', header=None).head()

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


### 7) mid_data_occupants.csv

#### 値

在室人員, 人

#### 例

In [11]:
pd.read_csv('core_data_tutorial/mid_data_occupants.csv', header=None).head()

,0,1,2
0,0.0,4.0,0.0
1,0.0,4.0,0.0
2,0.0,4.0,0.0
3,0.0,4.0,0.0
4,0.0,4.0,0.0


### 8) mid_data_q_trs_sol.csv

#### 値

透過日射熱取得[W]

#### 例

In [12]:
pd.read_csv('core_data_tutorial/mid_data_q_trs_sol.csv', header=None).head()

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


### 9) mid_data_theta_o_sol.csv

#### 値

外皮：裏面等価温度, ℃  
内壁の場合は裏面等価温度は0.0℃としている。

#### 例

In [13]:
pd.read_csv('core_data_tutorial/mid_data_theta_o_sol.csv', header=None).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.8000,0.8000,0.8000,0.8000,0.8000,0.8000,2.300,-4.45000,0.0,0.0,0.0,0.8000,0.8000,0.8000,0.8000,-4.45000,0.0,0.0,0.8000,0.8000,0.8000,2.300,0.0,0.8000,0.8000,0.8000,0.8000,0.8000,2.300,-4.45000,0.0,0.0
1,0.8875,0.8875,0.8875,0.8875,0.8875,0.8875,2.375,-4.31875,0.0,0.0,0.0,0.8875,0.8875,0.8875,0.8875,-4.31875,0.0,0.0,0.8875,0.8875,0.8875,2.375,0.0,0.8875,0.8875,0.8875,0.8875,0.8875,2.375,-4.31875,0.0,0.0
2,0.9750,0.9750,0.9750,0.9750,0.9750,0.9750,2.450,-4.18750,0.0,0.0,0.0,0.9750,0.9750,0.9750,0.9750,-4.18750,0.0,0.0,0.9750,0.9750,0.9750,2.450,0.0,0.9750,0.9750,0.9750,0.9750,0.9750,2.450,-4.18750,0.0,0.0
3,1.0625,1.0625,1.0625,1.0625,1.0625,1.0625,2.525,-4.05625,0.0,0.0,0.0,1.0625,1.0625,1.0625,1.0625,-4.05625,0.0,0.0,1.0625,1.0625,1.0625,2.525,0.0,1.0625,1.0625,1.0625,1.0625,1.0625,2.525,-4.05625,0.0,0.0
4,1.1500,1.1500,1.1500,1.1500,1.1500,1.1500,2.600,-3.92500,0.0,0.0,0.0,1.1500,1.1500,1.1500,1.1500,-3.92500,0.0,0.0,1.1500,1.1500,1.1500,2.600,0.0,1.1500,1.1500,1.1500,1.1500,1.1500,2.600,-3.92500,0.0,0.0


## Output

### 簡易版計算結果

In [14]:
pd.read_csv('../../docs/data_spec/spec_core_output_simple.csv')

,出力結果の種類,"列名 ([i]=室ID, [j]=境界ID）",単位,概要
0,室,rm[i]_ac_operate,－,運転状態
1,室,rm[i]_t_r,℃,空気温度
2,室,rm[i]_x_r,kg/kg(DA),室絶対湿度
3,室,rm[i]_ot,℃,室作用温度
4,室,rm[i]_l_s_c,W,対流空調顕熱負荷
5,室,rm[i]_l_s_r,W,放射空調顕熱負荷
6,室,rm[i]_l_l_c,W,対流空調潜熱負荷


### 詳細版計算結果

In [15]:
pd.read_csv('../../docs/data_spec/spec_core_output_detail.csv')

,出力結果の種類,"列名 ([i]=室ID, [j]=境界ID）",単位,概要
0,全体,out_temp,℃,外気温度
1,全体,out_abs_humid,kg/kg(DA),外気絶対湿度
2,室,rm[i]_ac_operate,－,運転状態
3,室,rm[i]_occupancy,boolean,在室状況
4,室,rm[i]_hc_hum,W/(m2･K),在室者周りの対流熱伝達率
5,室,rm[i]_hr_hum,W/(m2･K),在室者周りの放射熱伝達率
6,室,rm[i]_t_r,℃,空気温度
7,室,rm[i]_rh_r,%,室相対湿度
8,室,rm[i]_x_r,kg/kg(DA),室絶対湿度
9,室,rm[i]_mrt,℃,室MRT


## Example

In [16]:
ds, dd = core.calc(input_data_dir='core_data_tutorial')

KeyError: 'spec'

In [ ]:
ds.head()

In [ ]:
dd.head()